In [1]:
from OCC.Core.gp import gp_Pnt
from OCC.Core.Geom import Geom_BezierSurface, Geom_BSplineSurface, Geom_BezierCurve, Geom_BSplineCurve, Geom_Circle 
from OCC.Core.TColgp import TColgp_Array2OfPnt
from OCC.Core.TColStd import TColStd_Array1OfReal, TColStd_Array1OfInteger
from OCC.Core.ShapeAnalysis import ShapeAnalysis_FreeBounds
from OCC.Core.TopTools import TopTools_HSequenceOfShape
import json
import numpy as np

from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Ax2, gp_Vec, gp_Circ, gp_Lin, gp_Ax1, gp_Ax3, gp_Elips, gp_Hypr, gp_Parab, gp_Pln, gp_Cylinder, gp_Cone, gp_Sphere, gp_Torus, gp_Trsf
from OCC.Core.Geom import Geom_Circle, Geom_Line, Geom_BSplineCurve, Geom_TrimmedCurve, Geom_Ellipse, Geom_Hyperbola, Geom_Parabola, Geom_Plane, Geom_CylindricalSurface, Geom_ConicalSurface, Geom_SphericalSurface
from OCC.Core.GC import GC_MakeArcOfCircle, GC_MakeSegment, GC_MakeArcOfEllipse, GC_MakeArcOfHyperbola, GC_MakeArcOfParabola
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeFace ,BRepBuilderAPI_DisconnectedWire, BRepBuilderAPI_EmptyWire, BRepBuilderAPI_NonManifoldWire, BRepBuilderAPI_WireDone, BRepBuilderAPI_Transform
from OCC.Core.TopoDS import TopoDS_Edge
from OCC.Core.TopLoc import TopLoc_Location
from OCC.Core.Poly import Poly_Triangulation
from OCC.Core.TColgp import TColgp_Array1OfPnt
from OCC.Core.TColStd import TColStd_Array1OfReal, TColStd_Array1OfInteger
from OCC.Core.GeomAPI import GeomAPI_ProjectPointOnCurve
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Display.SimpleGui import init_display
from OCC.Core.BRepLib import breplib
from OCC.Core.ShapeFix import ShapeFix_Shape, ShapeFix_Wire, ShapeFix_Edge
from OCC.Core.ShapeExtend import ShapeExtend_DONE1
from OCC.Core.BRepCheck import BRepCheck_Analyzer
from OCC.Core.BRepOffsetAPI import BRepOffsetAPI_ThruSections
from OCC.Core.ShapeAnalysis import ShapeAnalysis_Wire
from OCC.Core.TopAbs import TopAbs_Orientation

In [3]:
from freecad_visualize_json_pythonocc import _build_and_mesh_face_robustly_from_topods, extract_mesh_from_face
from itertools import combinations

TypeError: 'type' object is not subscriptable

In [ ]:
def build_second_order_surface(face):
    surface_type = face['surface_type']
    parameter_range = face['parameter_range']
    u_min, u_max, v_min, v_max = parameter_range
    position = np.array(face['surface_definition']['position'])
    axis = np.array(face['surface_definition']['axis'])
    orientation = face['orientation']
    # 归一化参数范围
    if surface_type == 'Cylinder' or surface_type == 'Cone':
        position = position + axis * v_min
        v_max = v_max - v_min
        v_min = 0
    elif surface_type == 'Toriod':
        position = position + axis * u_min
        u_max = u_max - u_min
        u_min = 0
    print(position, axis, v_min, v_max)
    if surface_type == 'Cylinder':
        radius = face['surface_definition']['radius']
    elif surface_type == 'Cone':
        radius = face['surface_definition']['radius']
        semi_angle = face['surface_definition']['semi_angle']
    elif surface_type == 'Toroid':
        major_radius = face['surface_definition']['major_radius']
        minor_radius = face['surface_definition']['minor_radius']
    else:
        raise ValueError(f"Surface type {surface_type} not supported")
    
    occ_position = gp_Pnt(position[0], position[1], position[2])
    occ_axis_dir = gp_Dir(axis[0], axis[1], axis[2])
    
    # 2. 创建定义圆柱位置和方向的坐标系
    cylinder_ax3 = gp_Ax3(occ_position, occ_axis_dir)
    if surface_type == 'Cylinder':
        occ_surface = gp_Cylinder(cylinder_ax3, radius)
    elif surface_type == 'Cone':
        occ_surface = gp_Cone(cylinder_ax3, semi_angle, radius)
    elif surface_type == 'Toroid':
        occ_surface = gp_Torus(cylinder_ax3, major_radius, minor_radius)
    # 4. 使用BRepBuilderAPI_MakeFace根据参数范围裁剪几何体，生成面
    #    这是与C++ API最直接的对应
    face_builder = BRepBuilderAPI_MakeFace(occ_surface, u_min, u_max, v_min, v_max)
    # return face_builder.Face()
    rotation_axis = np.array(face['rotation_axis'])
    rotation_angle = face['rotation_angle']
    occ_rotation_axis = gp_Dir(rotation_axis[0], rotation_axis[1], rotation_axis[2])
    occ_rotation_center = occ_position
    rotation_axis = gp_Ax1(occ_rotation_center, occ_rotation_axis)
    trsf = gp_Trsf()
    # if surface_type == 'Cylinder' or surface_type == 'Cone':
    #     trsf.SetRotation(rotation_axis, 0)
    # elif surface_type == 'Toroid':
    trsf.SetRotation(rotation_axis, rotation_angle) # Bug: set rotation angle to 0
    transformer = BRepBuilderAPI_Transform(face_builder.Shape(), trsf)
    # print(type(transformer.Shape()))
    shape = transformer.Shape()
    mesher = BRepMesh_IncrementalMesh(shape, 0.1, True, 0.2)
    mesher.Perform() # 确保执行网格化

    if not mesher.IsDone():
         print("警告: BRepMesh_IncrementalMesh 执行后报告未完成，网格可能无效。")

    # --- 步骤 D: 提取网格数据 ---
    # 从修复并网格化后的面中提取数据
    vertices, faces = extract_mesh_from_face(shape)
    vertices = np.array(vertices)
    faces = np.array(faces)
    print(vertices.max(axis=0), vertices.min(axis=0))
    return shape, vertices, faces